In [ ]:
#plot&calc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.ticker as ticker
import umap

#ファイル管理系
import os
import glob

# Date time
import datetime

import scipy
import scipy.optimize as optimize
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Check data

In [ ]:
path_all_train_data_folder = "../0_Model_Construction_Protolith_Composition_Data/PRM_For_Metabasalt/"

# Compile data path
data_list = glob.glob(path_all_train_data_folder+"240509_Basalt_Compile/*.xlsx")
print(data_list)  

In [ ]:
data_compile = None
for data_path in data_list: # データの結合
    data_concat = pd.read_excel(data_path, header=7)
    if data_compile is None:
        data_compile = data_concat
    else:
        data_compile = pd.concat([data_concat, data_compile], axis=0)
data_compile = data_compile[data_compile["ANALYZED MATERIAL"]=="WHOLE ROCK"]
data_compile=data_compile.reset_index(drop=True)
data_compile.to_excel(path_all_train_data_folder+"240509_Basalt_Compile/Compile/Compiled.xlsx")

In [ ]:
#data_compile_ = data_compile.copy()
#data_compile = data_compile_.copy()
# data_compile = pd.read_excel(path_all_train_data_folder+"240509_Basalt_Compile/Compiled.xlsx")

In [ ]:
# CHECK rock type
data_compile["ROCK TYPE"].value_counts()

In [ ]:
# Select columns that are of type float64
data_float = data_compile.columns[data_compile.dtypes == "float64"].copy()

# Create a DataFrame with only the float columns
float_data_compile = data_compile[data_float].copy()

In [ ]:
drop_index=[]
################################################################################## Check and Drop Data
# Modified Section
# TiとAlを入れ間違えてる試料
Ti_Al_miss=data_compile[(float_data_compile["AL2O3"]<5)&(float_data_compile["TIO2"]>1)].copy()
iloc_values = [data_compile.index.get_loc(idx) for idx in Ti_Al_miss.index]
data_compile.loc[iloc_values, "TIO2"]=Ti_Al_miss["AL2O3"].values
data_compile.loc[iloc_values, "AL2O3"]=Ti_Al_miss["TIO2"].values

# MnとMgを入れ間違えてる試料
Mg_Mn_Miss=data_compile[(float_data_compile["AL2O3"]<5)&(float_data_compile["TIO2"]>1)&(float_data_compile["MGO"]<5)&(data_compile["REFERENCES"]=="TEASDALE, 2004")].copy()
iloc_values = [data_compile.index.get_loc(idx) for idx in Mg_Mn_Miss.index]
data_compile.loc[iloc_values, "MGO"]=Mg_Mn_Miss["MNO"].values
data_compile.loc[iloc_values, "MNO"]=Mg_Mn_Miss["MGO"].values

# MnとMgとCaを入れ間違えてる試料 paper
Mg_Mn_Ca_Miss = data_compile[(data_compile["REFERENCES"]=="LYONS, 2007")].copy()
iloc_values = [data_compile.index.get_loc(idx) for idx in Mg_Mn_Ca_Miss.index]
data_compile.loc[iloc_values, "MGO"]=Mg_Mn_Ca_Miss.loc[iloc_values, "CAO"]
data_compile.loc[iloc_values, "MNO"]=Mg_Mn_Ca_Miss["MGO"].values
data_compile.loc[iloc_values, "CAO"]=Mg_Mn_Ca_Miss["MNO"].values

# Ti not correct value
Ti_miss=data_compile[(data_compile["TIO2"]>9)].copy()
drop_index = list(drop_index)+list(Ti_miss.index.values)

# P obvious anomalous sample LE ROEX, 2010; WATSON, 2016; CASEY, 1997
P_miss=data_compile[(data_compile["P2O5"]>3)].copy()
drop_index = list(drop_index)+list(P_miss.index.values)

# Yb obvious anomalous sample 358→3.58
Yb_miss=data_compile[(data_compile["YB"]>100)].copy()
iloc_values = [data_compile.index.get_loc(idx) for idx in Yb_miss.index]
data_compile.loc[iloc_values, "YB"]=3.58

# PB obvious anomalous sample → reject reference→XRF only and many blank
Pb_miss=data_compile[(data_compile["PB"]>100)].copy()
drop_index = list(drop_index)+list(Pb_miss.index.values)

#FOURMENTRAUX, 2012→Anomalous in scatter plot in many element...
FOURMENTRAUX_anomaly=data_compile[(data_compile["REFERENCES"]=="FOURMENTRAUX, 2012")]
drop_index = list(drop_index)+list(FOURMENTRAUX_anomaly.index.values)

# Nb anomaly →CHAFFEY, 1989：Many data and cannot access paper, so accept.

# Mn anomaly →Most data do not have trace data, so reject for saving time
Yb_miss=data_compile[(data_compile["MNO"]>2.5)].copy()
iloc_values = [data_compile.index.get_loc(idx) for idx in Yb_miss.index]
data_compile.loc[iloc_values, "YB"]=3.58

# Li anomaly → mainly LONG, 2019 Anomalous MORB : accept

#  K2O: Most sample have large LOI (i.e., ~5-~10wt%) : Bigger than 3.5, reject; checked in 3, 3.5, 4
K2O_miss=data_compile[(data_compile["K2O"]>3.5)].copy()
drop_index = list(drop_index)+list(K2O_miss.index.values)

#  Cs, Vでも出てきたNGUYEN 2017のみAnomalous→Reject
Cs_miss=data_compile[(data_compile["CS"]>100)].copy()
drop_index = list(drop_index)+list(Cs_miss.index.values)

#  Ca Anomalous→Reject
Ca_miss=data_compile[(data_compile["CAO"]>100)].copy()
drop_index = list(drop_index)+list(Ca_miss.index.values)

# Na2O Anomalous→Reject
Na_miss=data_compile[(data_compile["NA2O"]>7)].copy()
drop_index = list(drop_index)+list(Na_miss.index.values)

# MNO Anomalous→Reject: containing sedimentary and most data analysed in XRF
Mn_miss=data_compile[(data_compile["MNO"]>2)].copy()
drop_index = list(drop_index)+list(Mn_miss.index.values)
################################################################################## Check and Drop Data
# まとめてdrop
data_compile = data_compile.drop(drop_index)

# data_compile["ROCK TYPE"] == "IGNEOUS:PLUTONIC:ULTRAMAFIC" # → Obviously anomaly in MGO →save in different file
data_compile_ULTRAMAFIC = data_compile[data_compile["ROCK TYPE"] == "IGNEOUS:PLUTONIC:ULTRAMAFIC"]

# UItramaficを除いたBasalticなデータ
data_compile_Basaltic = data_compile.drop(data_compile_ULTRAMAFIC.index)

In [ ]:
################################################################ USE FOR CHECKING DATA データの確認に使用
#P_miss=data_compile[(data_compile["MNO"]>2.5)].copy()
#P_miss.to_excel(path_all_train_data_folder+"Data Visualization/Check.xlsx")
################################################################ USE FOR CHECKING DATA データの確認に使用

In [ ]:
# 可視化
for elem in float_data_compile.columns:
    elem_list = ["SIO2", elem]
    data_use = data_compile[elem_list].dropna()
    sns.scatterplot(x=elem_list[0], y=elem_list[1], data=data_compile, hue="TECTONIC SETTING")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
    plt.savefig(path_all_train_data_folder+"/Data Visualization/Preprocessing_Before/"+elem+".png", bbox_inches="tight")
    plt.show()

In [ ]:
# 元素名の修正
############# prep
n=0
Major_list = ["SiO2", "Al2O3", "TiO2", "Fe2O3", "FeO", "MgO", "CaO", "K2O", "MnO", "P2O5", "Na2O", "LOI"]
Major_list_higher = ["SIO2", "AL2O3", "TIO2", "FE2O3", "FEO","MGO", "CAO", "K2O", "MNO", "P2O5", "NA2O", "LOI"]

############# prep

col_list_lower=[]
for col in data_compile_Basaltic.columns: # data_compile_Basaltic.columns[:18]は大きいまま
    if n<18: # 位置情報等の情報はそのまま
        pass
    elif col in Major_list_higher: # Major elementは変換
        #何番目のmajor elemかを確認
        index_num = Major_list_higher.index(col) 
        #変換先のmajor elem listから取得
        col = Major_list[index_num]
    else:
        #その他は2番目を小さくする
        col = col[:1] + col[1:].lower()
    n=n+1
    col_list_lower.append(col)
col_list_lower

In [ ]:
# 出力
data_compile_Basaltic.columns = col_list_lower
data_compile_Basaltic.to_excel(path_all_train_data_folder+"240509_Basalt_Compile/Compile/PetDB_Basaltic_"+str(datetime.date.today())+".xlsx")

data_compile_ULTRAMAFIC.columns = col_list_lower
data_compile_ULTRAMAFIC.to_excel(path_all_train_data_folder+"240509_Basalt_Compile/Compile/PetDB_ULTRAMAFIC_"+str(datetime.date.today())+".xlsx")

# 簡易可視化

In [ ]:
########################################################## UMAP Tectonic Setting
#UMAP_data_RAW = pd.read_excel(path_all_train_data, index_col=0, header=7)
UMAP_data_RAW = data_compile_Basaltic.copy()
#data_train_setting = pd.read_excel(path_all_train_data+"/Protolith/Primitive_not_applied.xlsx")

immobile_list = ["ZR", "TH", "TI", "NB"]

#使うデータを抽出
UMAP_data = UMAP_data_RAW[immobile_list].copy().dropna()#.apply(lambda x: np.log10(x)) # 使用データ log変換

############################################## UMAP 処理 前処理→UMAP→Compile
#####前処理 ->標準化　平均0　分散1
SDS = StandardScaler()
SDS.fit(UMAP_data)
UMAP_data_SD = SDS.transform(UMAP_data)

##### reducer
reducer = umap.UMAP(random_state=77)
reducer.fit(UMAP_data_SD)
#Protolith、SAの両方にフィット
#reducer.fit(np.concatenate([Protolith_data_SD, SA_protolith_SD]))
UMAP_data_umap = reducer.transform(UMAP_data_SD)

#####compile
UMAP_data_umap = pd.DataFrame(UMAP_data_umap, index = UMAP_data.index)
############################################## UMAP 処理 前処理→UMAP→Compile
####### read color value
fig, ax = plt.subplots(figsize=(10, 5))        
sns.scatterplot(data=UMAP_data_umap, x=0, y=1, palette="Dark2", alpha=1, ax=ax)
#plt.savefig(figure_folder_UMAP+'/umap_tectonic_setting.pdf', bbox_inches='tight')
plt.show()
########################################################## UMAP Tectonic Setting